In [1]:
import tensorflow as tf 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow.keras.layers as layers 
import time
import dataset
import glob

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
class Mappingnetwork(keras.layers.Layer):
    def __init__(self,outdim,**key):
        super(Mappingnetwork,self).__init__(**key)
        self.out_dim=outdim
    def build(self,input_shape):
        
        self.norm=[layers.BatchNormalization() for _ in range(9)]
        self.layers=[layers.Dense(self.out_dim)for i in range (8)]
        self.act=layers.LeakyReLU(0.2)
    def call(self,inputs,training=True):
        out=self.norm[0](inputs,training=training)
        for i in range(4):
            out1=self.layers[i*2](out)
            out2=self.act(out1)
            out=self.norm[i*2+1](out,training=training)
            out3=self.layers[i*2+1](out2)
            out4=self.act(out3)
            out4=self.norm[i*2+2](out4,training=training)
            out=out4+out
        return out 
    def get_config(self):
        return {"out_dim":self.out_dim}




In [3]:
class AdaIn(layers.Layer):
    def __init__(self,shape,**key):
        super(AdaIn,self).__init__(**key)
        self.shape=shape
    def build(self,input_shape):
        self.Dense=keras.layers.Dense(self.shape)
        self.act=keras.layers.LeakyReLU(0.2)
    def call(self,w,input):
        [m,b]=tf.nn.moments(input,axes=[1,2],keepdims=True)
        out1=input/b
        b=self.Dense(w)
        b=self.act(b)
        b=tf.expand_dims(b,1)
        b=tf.expand_dims(b,1)
        out=out1*b
        return out
    def get_config(self):
        return {"shape":self.shape}


In [4]:
class sub_sublayer(layers.Layer):
    def __init__(self,laten_dim):
        super(sub_sublayer,self).__init__()
        self.laten_dim=laten_dim
    def build(self,input_shape):
        self.adin=AdaIn(self.laten_dim)
        self.conv=layers.Conv2D(self.laten_dim,2,padding='same')
        self.act=layers.LeakyReLU(0.2)
    def call(self,input,w,b):
        out1=self.conv(input)
        out1=self.act(out1)
        b=tf.random.normal([tf.shape(w)[0],1,1,self.laten_dim])
        out2=out1+0.001*b
        out=self.adin(w,out2)
        return out
    def get_config(self):
        return {"laten_dim":self.laten_dim}


In [5]:
class style_layer(layers.Layer):
    def __init__(self,laten_dim,**key):
        super(style_layer,self).__init__(**key)
        self.laten_dim=laten_dim
    def build(self,input_shape):
        self.sub=[sub_sublayer(self.laten_dim) for i in range(2)]
        self.upsambel=layers.Conv2DTranspose(self.laten_dim,4,2,padding="same")
        self.act=layers.LeakyReLU(0.2)
    def call(self,input,w,b):
        out1=self.upsambel(input)
        out1=self.act(out1)
        out2=self.sub[0](out1,w,b)
        out3=self.sub[1](out2,w,b)
        return out3
    def get_config(self):
        return {"laten_dim":self.laten_dim}

    

In [6]:
class gen(layers.Layer):
    def __init__(self,laten_dim,**key):
        super(gen,self).__init__(**key)
        self.lat=laten_dim
    def build(self,input_shape):
        self.constant=tf.random.normal([1,4,3,self.lat])
        self.sub_layer=[style_layer(self.lat) for _ in range(5)]
        self.Dense=layers.Conv2D(3,3,activation="sigmoid",padding="same")
    def call(self,w,b):
        l1=tf.tile(self.constant,[tf.shape(w)[0],1,1,1])
        for i in range(5):
            l2=self.sub_layer[i](l1,w,b)
            l3=layers.UpSampling2D(2,interpolation='bilinear')(l1)
            l1=l2+l3
        l1=self.Dense(l1)
        return l1 
    def get_config(self):
        return {"laten_dim":self.lat}
    
    








In [7]:
class down_layer(layers.Layer):
    def __init__(self,laten_dim,**key):
        super(down_layer,self).__init__(**key)
        self.lat=laten_dim
    def build(self,input_shape):
        self.conv2=layers.Conv2D(self.lat,2,1,padding="same")
        self.down=layers.Conv2D(self.lat,4,2,padding="same")
        self.conv1=layers.Conv2D(self.lat,1,1,padding="same")
        self.act=layers.LeakyReLU(0.2)
    def call(self,input):
        l1=self.conv2(input)
        l1=self.act(l1)
        l2=self.down(l1)
        l2=self.act(l2)
        l3=self.conv1(l2)
        l3=self.act(l3)
        l4=layers.AveragePooling2D(2)(input)
        return l3+l4
    def get_config(self):
        return {"laten_dim":self.lat}

    


In [8]:
class discriminator(keras.Model):
    def __init__(self,laten_dim,**key):
        super(discriminator,self).__init__(**key)
        self.lat=laten_dim
    def build(self,input_shape):
        self.lays=[down_layer(self.lat) for i in range (5)]
        self.Dense0=keras.layers.Conv2D(self.lat,1,1,padding="same")
        self.Dense1=keras.layers.Dense(64)
        self.act=layers.LeakyReLU(0.2)
        self.Dense2=keras.layers.Dense(32)
        self.Dense3=keras.layers.Dense(1)
    def call(self,input):
        l1=self.Dense0(input)
        l1=self.act(l1)
        for i in range (5):
            l1=self.lays[i](l1)
        l2=layers.GlobalAveragePooling2D()(l1)
        l2=self.Dense1(l2)
        l2=self.act(l2)
        l2=self.Dense2(l2)
        l2=self.act(l2)
        l2=self.Dense3(l2)
        return l2
    def get_config(self):
        return {"laten_dim":self.lat}
    



In [9]:
input=keras.Input([64,])
Map1=Mappingnetwork(64)
gen1=gen(64)
l1=Map1(input)
l2=gen1(l1,None)
generator=keras.Model(input,l2)
discriminator_=discriminator(128)


In [15]:
# Instantiate one optimizer for the discriminator and another for the generator.
d_optimizer = keras.optimizers.RMSprop(learning_rate=0.0004)
g_optimizer = keras.optimizers.RMSprop(learning_rate=0.0005)

# Instantiate a loss function.
# loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)

latent_dim=64
@tf.function
def train_step(real_images):
    # Sample random points in the latent space
    for i in range(1):
        
        random_vector=tf.random.normal(shape=[real_images.shape[0],latent_dim])
        with tf.GradientTape() as tape1:
            with tf.GradientTape() as tape2: 
                generate_image_1=generator(random_vector)
                alph=tf.random.normal(shape=[real_images.shape[0],1,1,1])
                generate_image_2=alph*real_images+(1-alph)*generate_image_1
                out_real=discriminator_(real_images)
                out_gen_1=discriminator_(generate_image_1)
                out_gen_2=discriminator_(generate_image_2)
            grade_1=tape2.gradient(out_gen_2,generate_image_2)
            grade_l2=gradien_panalty_loss(grade_1)
            loss_dis=dis_loss(out_real,out_gen_1,grade_l2,20)
        grade_2=tape1.gradient(loss_dis,discriminator_.trainable_variables)
        d_optimizer.apply_gradients(zip(grade_2,discriminator_.trainable_variables))
    for i in range(1):
        random_vector=tf.random.normal(shape=[real_images.shape[0],latent_dim])
        with tf.GradientTape() as tape:
            generate_image= generator(random_vector)
            out=discriminator_(generate_image)
            loss_gen=gen_loss(out)
        grade=tape.gradient(loss_gen,generator.trainable_variables)
        g_optimizer.apply_gradients(zip(grade,generator.trainable_variables))
    return loss_dis,loss_gen, generate_image

In [11]:
@tf.function
def dis_loss(real,fake,grade,lamda):
    loss=tf.reduce_mean(fake)-tf.reduce_mean(real)+lamda*grade
    return loss
@ tf.function
def gen_loss(fake):
    return -tf.reduce_mean(fake) 
@ tf.function
def gradien_panalty_loss(grade):
    grad_sqr=tf.square(grade)
    grad_sqr_sum=tf.reduce_sum(grad_sqr,axis=np.arange(1,len(grad_sqr.shape)))
    grad_l2_norm=tf.sqrt(grad_sqr_sum)
    grad_penalty=tf.square(1-grad_l2_norm)
    return tf.reduce_mean(grad_penalty)


In [12]:
import os

# Prepare the dataset. We use both the training & test MNIST digits.
batch_size = 32
# (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
# all_digits = np.concatenate([x_train, x_test])
# all_label=keras.utils.to_categorical( np.concatenate([y_train,y_test]))
# all_digits = all_digits.astype("float32") / 255.0
# dataset = tf.data.Dataset.from_tensor_slices((all_digits,all_label))
img_path = glob.glob(r'E:/CelebA/Img/img_align_celeba/img_align_celeba/*.jpg')
dataset1, img_shape, _ = dataset.make_anime_dataset(img_path, batch_size,[128,96])



In [13]:
chekpoint=tf.train.Checkpoint(generator=generator,discriminator=discriminator_)
manager=tf.train.CheckpointManager(chekpoint,'./face_dis2',max_to_keep=3)

In [14]:
chekpoint.restore(tf.train.latest_checkpoint('./face_dis2'))
discriminator_=chekpoint.discriminator
generator=chekpoint.generator

In [20]:
# discriminator_.build(input_shape=[batch_size,128,96])
# generator.build(input_shape=[batch_size,128,96])
# discriminator_.set_weights(discriminator_1.get_weights())
# generator.set_weights(generator1.get_weights())

In [16]:

epochs = 2000       # In practice you need at least 20 epochs to generate nice digits.
save_dir = "./"
dataset1=dataset1.shuffle(1024)
for epoch in range(epochs):
    print("\nStart epoch", epoch)
    for step, real_images in enumerate(dataset1):
        # break
        # Train the discriminator & generator on one batch of real images.
        # rand_label1=keras.utils.to_categorical(np.random.randint(10,size=(batch_size,1)))
        # tf.random.set_seed(int(time.time()))
        d_loss, g_loss, generated_images= train_step(real_images)
        # print(real_images.shape)
        # Logging.
        if step % 200 == 0:
            # Print metrics
            print("discriminator loss at step %d: %.2f" % (step, d_loss))
            print("adversarial loss at step %d: %.2f" % (step, g_loss))

            # Save one generated image
            img = tf.keras.preprocessing.image.array_to_img(
                generated_images[0] * 255.0, scale=False
            )
            img.save(os.path.join(save_dir, "generated_img_gan2_" + str(step) + ".png"))
        #     # print(np.argmax(rand_label1,axis=-1)[0])
        # # To limit execution time we stop after 10 steps.
        # # Remove the lines below to actually train the model!
        # # if step > 10:
        # #     break
    print(manager.save())
        


oss at step 5600: -14.82
adversarial loss at step 5600: -265.54
discriminator loss at step 5800: -9.56
adversarial loss at step 5800: -97.39
discriminator loss at step 6000: -10.91
adversarial loss at step 6000: 309.86
discriminator loss at step 6200: 3.45
adversarial loss at step 6200: 177.94
./face_dis2\ckpt-1

Start epoch 1
discriminator loss at step 0: -15.93
adversarial loss at step 0: 83.68
discriminator loss at step 200: -1.71
adversarial loss at step 200: -152.53
discriminator loss at step 400: -13.01
adversarial loss at step 400: -2.05
discriminator loss at step 600: -24.10
adversarial loss at step 600: -144.45
discriminator loss at step 800: -16.81
adversarial loss at step 800: 114.23
discriminator loss at step 1000: -27.52
adversarial loss at step 1000: 68.78
discriminator loss at step 1200: 10.28
adversarial loss at step 1200: 3.54
discriminator loss at step 1400: -24.35
adversarial loss at step 1400: 37.93
discriminator loss at step 1600: -9.35
adversarial loss at step 160

KeyboardInterrupt: 